This project is an analysis of a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.  This analysis will incorporate cleaning the data as well as statistical analyses of the data in order to obtain some key information from it.

In [34]:
import numpy as np
import pandas as pd

In [35]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [36]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [37]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

This dataset contains a total of 50000 webpages from the German eBay, each representing a unique automobile.  We immediately notice five columns that contain null values, and all of those columns are of the 'object' datatype (i.e. likely a composite of multiple primitive datatypes like strings and integers), suggesting that handling those null values will not be as simple as merely replacing them with integers or strings; we may have to parse the information from the non-null values in those columns in order to get an idea of what to do with the nulls.

Let's first convert the column names from camelCase (to be more Pythonic) to snake_case and reword some of them to be more descriptive.

In [38]:
print(list(autos.columns))

['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'odometer', 'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen']


In [39]:
columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest', 'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model', 'odometer', 'registration_month', 'fuel_type', 'brand', 'unrepaired_damage', 'date_created', 'nr_of_pictures', 'postal_code', 'last_seen']

In [40]:
autos.columns = columns

In [41]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,date_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Let's investigate the dataframe to start finding starting points for cleaning it.

In [42]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,date_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 15:49:30,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The describe() function shows us a couple columns that need some scrutiny.  The registration_year column, for example, has a maximum year of 9999 (we are not even there yet) and a minimum year of 1000 (cars did not exist then).  Furthermore, the nr_of_pictures column contains nothing but 0s, and the 'offer_type' and 'seller' columns respectively contain only one value except for once, making them effectively useless and good candidates for dropping from the dataset.

Let's take a closer look at a few other columns that could be of interest for cleaning purposes, in particular the "odometer" and "price" columns.

In [43]:
autos['odometer'].value_counts()

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64

In [44]:
autos['price'].value_counts()

$0            1421
$500           781
$1,500         734
$2,500         643
$1,000         639
$1,200         639
$600           531
$3,500         498
$800           498
$2,000         460
$999           434
$750           433
$900           420
$650           419
$850           410
$700           395
$4,500         394
$300           384
$2,200         382
$950           379
$1,100         376
$1,300         371
$3,000         365
$550           356
$1,800         355
$5,500         340
$350           335
$1,250         335
$1,600         327
$1,999         322
              ... 
$72,600          1
$6,969           1
$1,310           1
$79,980          1
$2,128           1
$1,300,000       1
$8,720           1
$8,098           1
$20,589          1
$32,800          1
$7,050           1
$21,959          1
$1,275           1
$11,590          1
$38,400          1
$116,000         1
$1,775           1
$37,700          1
$5,485           1
$33,200          1
$3,620           1
$24,895     

In [45]:
autos['price'] = autos['price'].str.replace(',', '').str.replace('$', '').astype(float)
autos['price'].value_counts()

0.0           1421
500.0          781
1500.0         734
2500.0         643
1200.0         639
1000.0         639
600.0          531
800.0          498
3500.0         498
2000.0         460
999.0          434
750.0          433
900.0          420
650.0          419
850.0          410
700.0          395
4500.0         394
300.0          384
2200.0         382
950.0          379
1100.0         376
1300.0         371
3000.0         365
550.0          356
1800.0         355
5500.0         340
1250.0         335
350.0          335
1600.0         327
1999.0         322
              ... 
2225.0           1
69997.0          1
139997.0         1
69999.0          1
4780.0           1
8930.0           1
21599.0          1
15911.0          1
10000000.0       1
5180.0           1
919.0            1
1247.0           1
5998.0           1
27020.0          1
21888.0          1
46500.0          1
2001.0           1
2459.0           1
345000.0         1
34940.0          1
2785.0           1
5248.0      

In [46]:
autos['odometer'] = autos['odometer'].str.replace(',', '').str.replace('km', '').astype(float)
autos['odometer'].value_counts()

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
5000.0        967
40000.0       819
30000.0       789
20000.0       784
10000.0       264
Name: odometer, dtype: int64

In [47]:
autos = autos.rename(columns = {'odometer' : 'odometer_km'})
autos['odometer_km'].value_counts()

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
5000.0        967
40000.0       819
30000.0       789
20000.0       784
10000.0       264
Name: odometer_km, dtype: int64

In [48]:
autos['price'].value_counts().sort_index(ascending=True)

0.0           1421
1.0            156
2.0              3
3.0              1
5.0              2
8.0              1
9.0              1
10.0             7
11.0             2
12.0             3
13.0             2
14.0             1
15.0             2
17.0             3
18.0             1
20.0             4
25.0             5
29.0             1
30.0             7
35.0             1
40.0             6
45.0             4
47.0             1
49.0             4
50.0            49
55.0             2
59.0             1
60.0             9
65.0             5
66.0             1
              ... 
151990.0         1
155000.0         1
163500.0         1
163991.0         1
169000.0         1
169999.0         1
175000.0         1
180000.0         1
190000.0         1
194000.0         1
197000.0         1
198000.0         1
220000.0         1
250000.0         1
259000.0         1
265000.0         1
295000.0         1
299000.0         1
345000.0         1
350000.0         1
999990.0         1
999999.0    

The "odometer_km" column seems to contain expectable values, but the "price" column contains some zero prices as well as astronomically high prices, which should be removed.

In [49]:
autos = autos[autos['price'].between(1,360000)]
autos

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,date_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,158,andere,150000.0,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500.0,control,limousine,1997,automatik,286,7er,150000.0,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990.0,test,limousine,2009,manuell,102,golf,70000.0,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350.0,control,kleinwagen,2007,automatik,71,fortwo,70000.0,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350.0,test,kombi,2003,manuell,0,focus,150000.0,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
5,2016-03-21 13:47:45,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,privat,Angebot,7900.0,test,bus,2006,automatik,150,voyager,150000.0,4,diesel,chrysler,NaN,2016-03-21 00:00:00,0,22962,2016-04-06 09:45:21
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,privat,Angebot,300.0,test,limousine,1995,manuell,90,golf,150000.0,8,benzin,volkswagen,NaN,2016-03-20 00:00:00,0,31535,2016-03-23 02:48:59
7,2016-03-16 18:55:19,Golf_IV_1.9_TDI_90PS,privat,Angebot,1990.0,control,limousine,1998,manuell,90,golf,150000.0,12,diesel,volkswagen,nein,2016-03-16 00:00:00,0,53474,2016-04-07 03:17:32
8,2016-03-22 16:51:34,Seat_Arosa,privat,Angebot,250.0,test,NaN,2000,manuell,0,arosa,150000.0,10,NaN,seat,nein,2016-03-22 00:00:00,0,7426,2016-03-26 18:18:10
9,2016-03-16 13:47:02,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,privat,Angebot,590.0,control,bus,1997,manuell,90,megane,150000.0,7,benzin,renault,nein,2016-03-16 00:00:00,0,15749,2016-04-06 10:46:35


In [50]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,date_created,nr_of_pictures,postal_code,last_seen
count,48565,48565,48565,48565,48565.000000,48565,43979,48565.000000,46222,48565.000000,46107,48565.000000,48565.000000,44535,48565,39464,48565,48565.0,48565.000000,48565
unique,46882,37470,2,1,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,38474
top,2016-03-11 22:38:16,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,76,48564,48565,NaN,25019,12598,NaN,36102,NaN,3900,NaN,NaN,29368,10336,34775,1887,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,5888.935591,NaN,NaN,2004.755421,NaN,117.197158,NaN,125770.101925,5.782251,NaN,NaN,NaN,NaN,0.0,50975.745207,NaN
std,NaN,NaN,NaN,NaN,9059.854754,NaN,NaN,88.643887,NaN,200.649618,NaN,39788.636804,3.685595,NaN,NaN,NaN,NaN,0.0,25746.968398,NaN
min,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1200.000000,NaN,NaN,1999.000000,NaN,71.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30657.000000,NaN
50%,NaN,NaN,NaN,NaN,3000.000000,NaN,NaN,2004.000000,NaN,107.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49716.000000,NaN
75%,NaN,NaN,NaN,NaN,7490.000000,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71665.000000,NaN


In [51]:
autos['price'].value_counts()

500.0       781
1500.0      734
2500.0      643
1000.0      639
1200.0      639
600.0       531
800.0       498
3500.0      498
2000.0      460
999.0       434
750.0       433
900.0       420
650.0       419
850.0       410
700.0       395
4500.0      394
300.0       384
2200.0      382
950.0       379
1100.0      376
1300.0      371
3000.0      365
550.0       356
1800.0      355
5500.0      340
350.0       335
1250.0      335
1600.0      327
1999.0      322
400.0       321
           ... 
69993.0       1
4877.0        1
2225.0        1
69997.0       1
139997.0      1
69999.0       1
4780.0        1
8930.0        1
10790.0       1
2785.0        1
11240.0       1
898.0         1
5180.0        1
1247.0        1
5998.0        1
27020.0       1
21888.0       1
46500.0       1
2001.0        1
2459.0        1
345000.0      1
919.0         1
34940.0       1
5248.0        1
310.0         1
3129.0        1
69900.0       1
6202.0        1
18310.0       1
4349.0        1
Name: price, Length: 234

In [52]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

The 'date_crawled' column seems to have no special patterns other than that all of the data was collected between March and April of 2016.  April 6th and 7th had less collection than other days in the period.

In [53]:
autos['date_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

Similarly to the 'date_crawled' column, the 'date_created' column contains no apparent patterns.

In [54]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

The 'last_seen' column suggests that most of the webpages last saw activity within the final days of the web crawl, which would be expected.

Now we will start dealing with the 'registration_year' column and the discrepancies that we noted within it earlier.

In [55]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Let's clean our dataset so that it contains only cars with registration years between 1900 (a reasonably flexible starting date) and the current year, 2018.

In [57]:
autos = autos[autos['registration_year'].between(1900,2019)]

In [58]:
autos['registration_year'].value_counts(normalize=True).sort_index(ascending=True)

1910    0.000103
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000041
1937    0.000082
1938    0.000021
1939    0.000021
1941    0.000041
1943    0.000021
1948    0.000021
1950    0.000062
1951    0.000041
1952    0.000021
1953    0.000021
1954    0.000041
1955    0.000041
1956    0.000082
1957    0.000041
1958    0.000082
1959    0.000124
1960    0.000474
1961    0.000124
1962    0.000082
1963    0.000165
1964    0.000247
1965    0.000350
1966    0.000453
1967    0.000536
1968    0.000536
          ...   
1990    0.007148
1991    0.006983
1992    0.007622
1993    0.008755
1994    0.012957
1995    0.025276
1996    0.028283
1997    0.040190
1998    0.048676
1999    0.059677
2000    0.065012
2001    0.054300
2002    0.051210
2003    0.055598
2004    0.055680
2005    0.060480
2006    0.055001
2007    0.046905
2008    0.045628
2009    0.042950
2010    0.032733
2011    0.033433
2012    0.026985
2013    0.016541
2014    0.013657
2015    0.008075
2016    0.025131
2017    0.0286

Now let's try to get some trends for each car brand.

In [61]:
autos['brand'].value_counts()

volkswagen        10331
opel               5274
bmw                5274
mercedes_benz      4650
audi               4168
ford               3382
renault            2325
peugeot            1430
fiat               1262
seat                919
skoda               780
nissan              741
mazda               739
smart               694
citroen             685
toyota              611
hyundai             483
sonstige_autos      468
volvo               439
mini                418
mitsubishi          397
honda               388
kia                 345
alfa_romeo          321
porsche             287
suzuki              286
chevrolet           275
chrysler            169
dacia               129
daihatsu            122
jeep                107
subaru              102
land_rover           99
saab                 79
daewoo               76
jaguar               74
trabant              67
rover                65
lancia               55
lada                 29
Name: brand, dtype: int64

We can see that German brands are the most popular in their home country, with a German brand occupying each of the top five slots.  Let's see how their pricing differs from all other brands that have at least 1% market share.

In [85]:
brand_prices = {}
for brand in autos['brand'].value_counts().index:
    if autos['brand'].value_counts()[brand] >= len(autos)*0.01:
        brand_prices[brand] = autos[autos['brand'] == brand]['price'].mean()

In [86]:
brand_prices

{'audi': 9212.9306621881,
 'bmw': 8261.382442169132,
 'citroen': 3756.07299270073,
 'fiat': 2793.8700475435817,
 'ford': 3728.4121821407452,
 'mazda': 4059.059539918809,
 'mercedes_benz': 8526.623225806452,
 'nissan': 4669.3859649122805,
 'opel': 2941.4664391353813,
 'peugeot': 3065.611888111888,
 'renault': 2431.195698924731,
 'seat': 4320.168661588684,
 'skoda': 6353.544871794872,
 'smart': 3518.102305475504,
 'toyota': 5148.0032733224225,
 'volkswagen': 5333.1962055948115}

This dictionary shows that the average price of German cars tends to be at least a bit higher than that of foreign brands; other than Opel, which is a low cost German manufacturer, the lowest average German price is 5333, whereas the highest average non-German  price is 6353, and the German prices extend to over 9000.

Now let's do the same for mileage.  Later, we will use both our mean price dictionary and our mean mileage dictionary to construct a dataframe comparing the two sets of data.

In [89]:
brand_mileage = {}
for brand in autos['brand'].value_counts().index:
    if autos['brand'].value_counts()[brand] >= len(autos)*0.01:
        brand_mileage[brand] = autos[autos['brand'] == brand]['odometer_km'].mean()
brand_mileage

{'audi': 129492.56238003839,
 'bmw': 132682.97307546454,
 'citroen': 120160.58394160584,
 'fiat': 117567.35340729002,
 'ford': 124349.49733885274,
 'mazda': 124871.44790257105,
 'mercedes_benz': 130848.3870967742,
 'nissan': 118711.20107962213,
 'opel': 129452.02882062951,
 'peugeot': 127356.64335664336,
 'renault': 128062.36559139784,
 'seat': 121768.22633297062,
 'skoda': 110993.58974358975,
 'smart': 100511.52737752162,
 'toyota': 116219.31260229132,
 'volkswagen': 128955.570612719}

Now to create a convenient dataframe out of both dictionaries for comparison purposes.

In [90]:
bm_series = pd.Series(brand_mileage)
bp_series = pd.Series(brand_prices)
bpbm_df = pd.DataFrame(bp_series, columns=['mean_price'])
bpbm_df['mean_mileage'] = bm_series

In [91]:
bpbm_df

,mean_price,mean_mileage
audi,9212.930662,129492.562380
bmw,8261.382442,132682.973075
citroen,3756.072993,120160.583942
fiat,2793.870048,117567.353407
ford,3728.412182,124349.497339
mazda,4059.059540,124871.447903
mercedes_benz,8526.623226,130848.387097
nissan,4669.385965,118711.201080
opel,2941.466439,129452.028821
peugeot,3065.611888,127356.643357


This comparison dataframe shows two things: 1) The average mileage does not change drastically from brand to brand, but 2) the average price does swing significantly, with luxury German brands showing the highest ratio of cost per mile.